# Asistente de soporte inteligente para una empresa de tecnología

Una empresa de software llamada TechNova tiene miles de tickets de soporte de sus clientes acumulados en su sistema.

Cada ticket contiene:

- El texto de la consulta del cliente.

- El tipo de problema (categoría: instalación, conexión, errores, licencias, etc.).

- Y una respuesta técnica redactada por los agentes.

El departamento de soporte quiere crear un asistente interno inteligente que, al recibir una nueva consulta, recomiende los tickets anteriores más parecidos en significado (aunque usen palabras diferentes), para ayudar a los agentes a responder más rápido.

## Objetivo general

Desarrollar un programa en Python que:

1. Carga el archivo csv de tickets.

2. Genera embeddings para cada ticket (texto de problema).

3. Almacena en una base de datos vectorial.

4. Permite ingresar una nueva consulta del cliente.

5. Devuelve los tickets más similares en significado, junto con su respuesta original.

In [1]:
#1. Cargar el archivo CSV de tickets
import pandas as pd
tickets_df = pd.read_csv('Data/tickets.csv')
# Visualizar los primeros registros
tickets_df.head()

,id,categoria,problema,respuesta
0,1,Instalación,No puedo instalar el software en Windows 11,Verifica que tengas permisos de administrador ...
1,2,Conexión,El programa no se conecta a la base de datos,Revisa el puerto y las credenciales en el arch...
2,3,Error,Aparece un mensaje 'Invalid License Key' al ab...,Asegúrate de que la licencia esté activa y vin...
3,4,Rendimiento,El sistema se vuelve lento después de varias h...,Reinicia el servicio y limpia la caché temporal.
4,5,Instalación,La instalación se detiene al 50% sin error vis...,Cierra otros programas y vuelve a ejecutar el ...


In [2]:
#2. Generar embeddings de los tickets
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Debemos generarlos para cumplir con: "recomiende los tickets anteriores más parecidos en significado (aunque usen palabras diferentes), para ayudar a los agentes a responder más rápido."
embeddings = model.encode(tickets_df['problema'].tolist(), convert_to_tensor=True)
tickets_df['embedding'] = embeddings.tolist()



c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#3. Almacenar los embeddings en una base de datos vectorial
import chromadb
client = chromadb.Client()
collection = client.create_collection(name="tickets_collection")
for idx, row in tickets_df.iterrows():
    collection.add(
        documents=[row['problema']],
        metadatas=[{
            'ticket_id': int(row['id']), 
            'categoria': row['categoria'],
            'respuesta': row['respuesta']
        }],
        ids=[str(row['id'])],
        embeddings=[row['embedding']]
    )

In [5]:
#4. Consultas de los clientes
query = input("Ingrese la descripción del problema: ")
query_embedding = model.encode([query])
results = collection.query(
    query_embeddings=query_embedding.tolist(),
    n_results=5
)


In [6]:
#5. Mostrar resultados: tickets más similares en significado, junto con respuesta
print(f"\nResultados para: '{query}'\n")
for i in range(len(results['ids'][0])):
    ticket_id = results['ids'][0][i]
    problema = results['documents'][0][i]
    metadata = results['metadatas'][0][i]
    
    print(f"Ticket ID: {ticket_id}")
    print(f"Categoría: {metadata['categoria']}")
    print(f"Problema: {problema}")
    print(f"Respuesta: {metadata['respuesta']}")
    print("-" * 80)



Resultados para: 'no se conecta a la base de datos'

Ticket ID: 2
Categoría: Conexión
Problema: El programa no se conecta a la base de datos
Respuesta: Revisa el puerto y las credenciales en el archivo de configuración.
--------------------------------------------------------------------------------
Ticket ID: 12
Categoría: Conexión
Problema: No puedo sincronizar mis datos entre dispositivos
Respuesta: Asegúrate de estar conectado a internet y de usar la misma cuenta en ambos dispositivos.
--------------------------------------------------------------------------------
Ticket ID: 4
Categoría: Rendimiento
Problema: El sistema se vuelve lento después de varias horas
Respuesta: Reinicia el servicio y limpia la caché temporal.
--------------------------------------------------------------------------------
Ticket ID: 16
Categoría: Actualización
Problema: La actualización no descarga correctamente
Respuesta: Comprueba tu conexión a internet y vuelve a intentar la actualización manualmente.

## Herramientas y tecnologías sugeridas

Python 3.10+

VS Code

Librerías:


In [ ]:
pip install chromadb sentence-transformers pandas

## Preguntas

1. Diseño del vector

- ¿Qué campos del ticket deben incluirse en el texto del embedding (solo “problema” o también “categoría”)?

- ¿Cómo afectaría esto los resultados?

2. Elección de métrica

- ¿Por qué es mejor usar cosine similarity que distancia euclidiana para texto?

3. Optimización

- ¿Qué pasaría si hay 100 000 tickets?

- ¿Qué tipo de indexación sería más eficiente (HNSW, IVF, PQ)?

4. Interpretación

- Si los tres tickets más similares pertenecen a la misma categoría, ¿qué conclusión se puede sacar?

5. Ampliación (opcional)

- ¿Cómo integrarían este sistema en una interfaz web o chatbot?